# Complete Sentiment Analysis
This requires a complete async sentiment analysis using comprehend.

Ouput's a dataframe the same as objects, with objects that have been analysed by comprehend.
The dataframe will have additional columns: aws_input_line_number, language_sentiment, language_sentiment_scores_*.
`aws_input_line_numbers`: the line number that the object had for the input file.
`language_sentiment`: `POSTIVE|NEUTRAL|NEGATIVE`, sentiment as given by Comprehend.
`language_sentiment_score_[mixed|neutral|positive|negative]`: Score for each class as probability according to AWS comprehend.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag.third_party_models.aws_async import complete_sentiment, info_sentiment
from phoenix.tag.third_party_models.aws_async import job_types

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

# TODO: this has to be refactored so that MONTH_PREFIX is relational to the YEAR_FILTER and MONTH_FILTER
YEAR_FILTER = 2021
MONTH_FILTER = 5
MONTH_PREFIX = f"year_filter={YEAR_FILTER}/month_filter={MONTH_FILTER}/"

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/facebook_posts/{MONTH_PREFIX}"

# Input
ASYNC_JOB_GROUP = artifacts.json.url(ARTIFACTS_BASE_URL, "async_job_group")

# Output
LANGUAGE_SENTIMENT_OBJECTS = artifacts.dataframes.url(ARTIFACTS_BASE_URL, "language_sentiment_objects")

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
RUN_DATE,
ASYNC_JOB_GROUP,
LANGUAGE_SENTIMENT_OBJECTS,
sep='\n',
)

In [ ]:
async_job_group = job_types.get_json(ASYNC_JOB_GROUP)
async_job_group

In [ ]:
job_infos = info_sentiment.get_job_infos(async_job_group)
job_infos

In [ ]:
info_sentiment.are_processable_jobs(job_infos)

In [ ]:
language_sentiment_objects = complete_sentiment.complete_sentiment_analysis(async_job_group, job_infos)

In [ ]:
language_sentiment_objects.head()

In [ ]:
language_sentiment_objects.shape

In [ ]:
a = artifacts.dataframes.persist(LANGUAGE_SENTIMENT_OBJECTS, language_sentiment_objects)